In [ ]:
class AES:

# 1. BẢNG TRA S-BOX (Substitution Box)    
    S_BOX = (
        0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
        0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
        0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
        0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
        0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
        0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
        0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
        0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
        0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
        0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
        0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
        0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
        0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
        0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
        0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
        0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16,
       )
    
# 2. HẰNG SỐ R-CON (Dùng cho Key Expansion)
    R_CON = (
        0x00, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40,
        0x80, 0x1B, 0x36, 0x6C, 0xD8, 0xAB, 0x4D, 0x9A,
        0x2F, 0x5E, 0xBC, 0x63, 0xC6, 0x97, 0x35, 0x6A,
        0xD4, 0xB3, 0x7D, 0xFA, 0xEF, 0xC5, 0x91, 0x39,
    )
    @staticmethod
    def _xtime(a):
        # Nhân đôi trong trường hữu hạn GF(2^8)
        # Nếu bit cao nhất (0x80) là 1 -> Dịch trái rồi XOR 0x1B
        # Nếu không -> Chỉ dịch trái
        return (((a << 1)^ 0x1B) & 0xFF) if (a & 0x80) else (a << 1)


    def _bytes2matrix(self, text):
        #Chuyển đổi 16 bytes thành ma trận 4x4 (List of lists)
        return [list(text[i:i+4]) for i in range(0, len(text), 4)]

    def _matrix2bytes(self, matrix):
        # chuyển đổi ma trận thành dạng 16 bytes
        return bytes(sum(matrix, []))
    
    def _sub_bytes(self, state):
        # thay thế từng byte bằng S_BOX
        for i in range(4):
            for j in range(4):
                state[i][j] = self.S_BOX[state[i][j]]

    def _shift_rows(self, state):
        # dịch các hàng (sang trái)
        state[1][:] = state[1][1:] + state[1][:1]
        state[2][:] = state[2][2:] + state[2][:2]
        state[3][:] = state[3][3:] + state[3][:3]
        
    def _mix_columns(self, state):
        # Trộn cột sử dụng toán học GF(2^8) tối ưu
        for i in range(4):
            c0, c1, c2, c3 = state[0][i], state[1][i], state[2][i], state[3][i] # 4 byte cua 1 cot\
                
            # tổng XOR của cả cột
            t = c0^ c1^ c2^ c3
            
            # Công thức nhân Galois: i XOR t XOR xtime( i XOR i+1)
            state[0][i] ^= t ^ self._xtime(c0 ^ c1)
            state[1][i] ^= t ^ self._xtime(c1 ^ c2)
            state[2][i] ^= t ^ self._xtime(c2 ^ c3)
            state[3][i] ^= t ^ self._xtime(c3 ^ c0)
        
    def _add_round_key(self, state, round_key):
        for i in range(4):
            for j in range(4):
                state[i][j] ^= round_key[i][j]
                
    def _expand_key(self, master_key):  #sinh cac khoa con
        #khoi tao  = khoa goc
        key_columns = self._bytes2matrix(master_key)
        
        i = 1
        
        # vong lap de sinh 11 khoa va 44 cot( 11*44)
        while len(key_columns) < 44:
            word = list(key_columns[-1])
            
            # neu la dong dau tien cua khoa moi
            if len(key_columns) % 4 == 0:
                
                #rotword
                word.append(word.pop(0))
                #sub_bytes
                word = [self.S_BOX[b] for b in word]
                # XOR voi Rcon( chi byte dau tien)
                word[0] ^= self.R_CON[i]
                i += 1
            prev_word = key_columns[-4]
            #zip() de khop word voi prev, ghep tung cap roi XOR voi nhau
            word = [w ^ p for w, p in zip(word, prev_word)]
            key_columns.append(word)
            
        return [key_columns[4*k : 4*(k+1)] for k in range (11)]
    
    def __init__(self, master_key: bytes):
        
        # khoi tao class va sinh khoa con
        assert len(master_key) == 16
        self.round_keys = self._expand_key(master_key)
        pass
    
    def encrypt_block(self, plaintext):
        assert len(plaintext) == 16
        
        # 1. Chuyển input bytes thành ma trận state
        state = self._bytes2matrix(plaintext)

        # 2. Round 0: Chỉ AddRoundKey
        self._add_round_key(state, self.round_keys[0])

        # 3. Round 1 -> 9: Lặp 9 lần quy trình chuẩn
        for i in range(1, 10):
            self._sub_bytes(state)
            self._shift_rows(state)
            self._mix_columns(state) # Bước này làm xáo trộn mạnh nhất
            self._add_round_key(state, self.round_keys[i])

        # 4. Round 10: Vòng cuối KHÔNG CÓ MixColumns
        self._sub_bytes(state)
        self._shift_rows(state)
        self._add_round_key(state, self.round_keys[10])

        # 5. Chuyển ma trận kết quả về dạng bytes
        return self._matrix2bytes(state)

def pad_data(data_str):
    data_bytes = data_str.encode('utf-8')
    pad_len = 16 - (len(data_bytes) % 16)
    # Ví dụ: GIAOTHONGVANTAI (15 bytes) -> thiếu 1 -> thêm \x01
    padding = bytes([pad_len] * pad_len)
    return data_bytes + padding

def pad_key(key_str):
    key_bytes = key_str.encode('utf-8')
    if len(key_bytes) < 16:
        key_bytes += b'\x00' * (16 - len(key_bytes))
    return key_bytes[:16]

# --- VÍ DỤ CHẠY THỰC TẾ ---
if __name__ == "__main__":
    
    # 1. Dữ liệu đầu vào
    raw_key = "TRANDUCLINH"
    raw_text = "GIAOTHONGVANTAI"
    
    # 2. Chuẩn hóa dữ liệu (Padding)
    key_16b = pad_key(raw_key)
    text_16b = pad_data(raw_text)
    
    print(f"Key gốc      : {raw_key}")
    print(f"Key (Hex)    : {key_16b.hex().upper()}")
    print("-" * 30)
    print(f"Plaintext    : {raw_text}")
    print(f"Input (Hex)  : {text_16b.hex().upper()}")
    
    # 3. Khởi tạo và Mã hóa
    my_aes = AES(key_16b)
    ciphertext = my_aes.encrypt_block(text_16b)
    
    print("-" * 30)
    print(f"Ciphertext   : {ciphertext.hex().upper()}")

Key gốc      : TRANDUCLINH
Key (Hex)    : 5452414E4455434C494E480000000000
------------------------------
Plaintext    : GIAOTHONGVANTAI
Input (Hex)  : 4749414F54484F4E4756414E54414901
------------------------------
Ciphertext   : C9FBB9288919902E91FD9063D8AC2D05
